## Story

#### Sur le port d’Amsterdam, Alex est responsable du remplissage des conteneurs. Il doit donc organiser le placement de boites de différentes dimensions dans des conteneurs. Pour se faire, Alex met à profit son expérience pour placer au mieux les différentes boites et minimiser le nombre total de conteneurs utilisés. Cette méthode n'étant pas optimale, Alex fait appel à CapGemini pour créer une application afin d'identifier et visualiser la solution optimale de rangement des conteneurs. Grace à l’application, Alex n’a plus qu’a renseigner les quantités et dimensions de chaque boite pour que l’algorithme lui propose l’arrangement utilisant le minimum de conteneurs. Il a également la possibilité de visualiser le chargement virtuel de chaque conteneur pour mieux appréhender le remplissage réel de ce dernier. 

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import random

import plotly
import plotly.graph_objects as go

/tmp/ipykernel_5613/3361066208.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipykernel_5613/3361066208.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


## Algorithm

#### Class of used object

In [2]:
class Box:
    
    def __init__(self, w, d, h):
        self.dimensions = {'w':w, 'd':d, 'h':h}
        self.v = w * d * h
        self.position = {'x':0, 'y':0, 'z':0} #x ,y ,z   (width, depth, height)
    
    def get_w(self):
        return self.dimensions.get('w')
    
    def get_d(self):
        return self.dimensions.get('d')
    
    def get_h(self):
        return self.dimensions.get('h')
    
    def get_dimensions_wdh(self):
        return self.dimensions['w'], self.dimensions['d'], self.dimensions['h']
    
    def get_x(self):
        return self.position.get('x')
    
    def get_y(self):
        return self.position.get('y')
    
    def get_z(self):
        return self.position.get('z')
        
    def __repr__(self):
        return "w:{}, d:{}, h:{} v:{}".format(self.get_w(), self.get_d(), self.get_h(), self.v)
    
class Item(Box):
    
    def __init__(self, w, d, h, name = ""):
        Box.__init__(self, w, d, h)
        self.rotation = "XYZ"
        self.name = name  
    
    def __repr__(self):
        return "{} w:{}, d:{}, h:{} v:{} rotation:{} position:{}".format(self.name, self.get_w(),\
                                                                      self.get_d(), self.get_h(), self.v,\
                                                                      self.rotation, self.position )

class Container(Box):
    
    def __init__(self, w, d, h):
        Box.__init__(self, w, d, h)
        self.v_fill = 0 
        self.pivots = [{'x':0, 'y':0, 'z':0}] 
        self.items = []
        self.items_names_already_checked = []
        
    def get_volume_empty(self):
        return self.v - self.v_fill
    
    def __repr__(self):
        return "w:{}, d:{}, h:{} v:{} contains {} item(s) which fill {} % of volume".format(self.get_w(), self.get_d(), self.get_h(), self.v, len(self.items), round(self.v_fill *100/self.v , 2))

In [3]:
rotations = {"XYZ": 'XYZ', "YXZ": 'YXZ',
  "ZYX": 'ZYX', "YZX": 'YZX',
  "XZY": 'XZY', "ZXY": 'ZXY'}

#### Functions

In [4]:
def add_new_container(containers, width, depth, length):
    """Add a container with dimensions (width, depth, length) to list containers
    
    keyword arguments:
    width -- the width of the container that you want add
    depth -- the depth of the container that you want add
    length -- the length of the container that you want add
    """
    container = Container(width, depth, length)  #Container(12032, 2698, 2350)
    containers.append(container)

In [5]:
def get_dimensions_item(item, rotation):
    """Get dimension of the item depending on his dimensions
    
    keyword arguments:
    item -- instance of Item that you want dimension
    rotation -- String meaning rotation
    """
    if rotation == "XYZ":
        return {"w": item.get_w(), "d": item.get_d(), "h": item.get_h()}
    elif rotation == "YXZ":
        return {"w": item.get_d(), "d": item.get_w(), "h": item.get_h()}
    elif rotation == "ZYX":
        return {"w": item.get_h(), "d": item.get_d(), "h": item.get_w()}
    elif rotation == "YZX":
        return {"w": item.get_d(), "d": item.get_h(), "h": item.get_w()}
    elif rotation == "XZY":
        return {"w": item.get_w(), "d": item.get_h(), "h": item.get_d()}
    elif rotation == "ZXY":
        return {"w": item.get_h(), "d": item.get_w(), "h": item.get_d()}
    else:
        return {"w": item.get_w(), "d": item.get_d(), "h": item.get_h()}

#### Function to apply constraints

In [6]:
def is_overlap(item1, position_item1, rotation_item1, item2, position_item2, rotation_item2):
    """If 2 both items are sticked (share same coordinates to extremities), return False
    Meaning its not considering  to an overlap
    
    keyword arguments:
    item1, item2 -- instance of Item that you want to know if this is overlapped or not
    position_item1, position_item2 -- Dictionnary that they have position (x,y,z)
    rotation_item1, rotation_item2 -- String meaning rotation
    """
    dimensions_item1 = get_dimensions_item(item1, rotation_item1)
    dimensions_item2 = get_dimensions_item(item2, rotation_item2)
    w1, d1, h1 = (dimensions_item1.get('w'), dimensions_item1.get('d'), dimensions_item1.get('h'))
    w2, d2, h2 = (dimensions_item2.get('w'), dimensions_item2.get('d'), dimensions_item2.get('h'))
    
    x1, y1 ,z1 = (position_item1.get('x'), position_item1.get('y'), position_item1.get('z'))
    x2, y2 ,z2 = (position_item2.get('x'), position_item2.get('y'), position_item2.get('z'))
    
    #Condition pour un non overlap ==> on retourne le contraire pour savoir si il y a un overlap :)
    condition_non_overlap = (x1+w1 <= x2) or (x2+w2 <= x1) or (y1+d1 <= y2) or (y2+d2 <= y1) or (z1+h1 <= z2) or (z2+h2 <= z1)
    return (not condition_non_overlap)

def is_legit(item_to_check, position_item_to_check, rotation_item_to_check, item_to_deal_with, position_item_to_deal_with, rotation_item_to_deal_with):
    """If the item to check fits with the item to deal with return True. Otherwise return False.
    If the item to check is not a problem for other items it's OK. For further, it's here we apply constraint """
    overlap_cond = is_overlap(item_to_check, position_item_to_check, rotation_item_to_check, item_to_deal_with, position_item_to_deal_with, rotation_item_to_deal_with)
    
    return overlap_cond

In [7]:
def put_item(container, item): #Vérification des contraintes
    """Return true if the item fit in the container
    
    keyword arguments:
    container -- instance of Container that you want to put the item in
    item -- instance of Item that you want put in the container
    """
    global rotations
    
    for position in container.pivots:
        for rotation_key in rotations:
            dimensions_item_rotated = get_dimensions_item(item, rotation_key)

            # Try until space is found
            if container.get_w() < position['x'] + dimensions_item_rotated['w'] or \
                container.get_d() < position['y'] + dimensions_item_rotated['d'] or \
                container.get_h() < position['z'] + dimensions_item_rotated['h']: 
                continue

            # Check if there are any intersections
            intersects = any(list(map(lambda x : is_legit(item, position, rotation_key, x, x.position, x.rotation), container.items)))
            
            # If there are none, we can assign this bag to this container
            if not intersects:
                ##We set information to item
                item.position = position   
                item.rotation = rotations[rotation_key]
                container.v_fill += item.v
                
                container.pivots.remove(position)  ##we remove that we just use
                container.pivots.extend(get_pivots(item))
                
                #Bag is cloned to ensure correct data storage
                container.pivots.sort(key=lambda pivot: (pivot['z'],pivot['x'],pivot['y']))
                container.items.append(item)
                return True

    return False

In [8]:
def get_pivots(item):
    """Return pivots of the item that you give in parameter. Pivots are:
        -Corner back down right
        -Corner front down left
        -Corner back upper left
    
    keyword arguments:
    item -- instance of Item that you want pivots
    """
# Try to find availble pivot point for each item
    pivots = []
    ##On va mettre les dimensions de l'objet rotationné dans les pivots
    dim_item_rotation = get_dimensions_item(item, item.rotation)
    w, d, h = dim_item_rotation.get("w"), dim_item_rotation.get("d"), dim_item_rotation.get("h")
    pivots.append({"x": item.get_x() + w, "y": item.get_y(), "z": item.get_z()})
    pivots.append({"x": item.get_x(), "y": item.get_y() + d, "z": item.get_z()})
    pivots.append({"x": item.get_x(), "y": item.get_y(), "z": item.get_z() + h})
    return pivots

##### Core functions

In [9]:
def best_fit_3D(items_not_packed, container_wdh, containers_with_items = None):
    """Return a list of Container with dimensions container_wdh with items_not_packed inside.
    Use 3D first fit decreasing algorithm. If containers_with_items the algorithm will not start from
    scratch but it will use this list
    
    keyword arguments:
    items_not_packed -- list of Item that you want pack
    container_wdh -- tuple of dimension of the container that you want for the container (width, depth, height)
    containers_with_items -- list of containers that already contains items
    """
    
    if containers_with_items is None:
        containers = []
        first_add_skip = False 
    else:
        containers = containers_with_items
        first_add_skip = True 
    
    is_item_too_big = False
    
    ##Sort for decreasing :) 
    items_not_packed.sort(key = lambda item: item.v, reverse = True)
    
    while len(items_not_packed) != 0:   #Boucle do while a ==> do while useless 🙃
        print(len(items_not_packed))
        to_pack = items_not_packed
        items_not_packed = [] ##Clear not packed
        
        #Will not add a box in the beginning if we put a list in parameter
        if not first_add_skip:
#             add_new_container(containers, 12032, 2698, 2350)
            add_new_container(containers, container_wdh[0], container_wdh[1], container_wdh[2])
    
        first_add_skip = False
        
        axis = {"x":"x", "y":"y", "z":"z"}
    
        for num_box_to_pack in range(len(to_pack)):
            current_item = to_pack[num_box_to_pack]
            fitted = False
            
            #We try to fit the box for each container and pivot 
            ### Check si on a besoin de d'itérer sur tous. Pk pas prendre le container nouvellement créer ?🔨🔨🔨🔨🔨🔨
            for container in containers:
                
                if current_item.v > container.v or max(current_item.get_dimensions_wdh()) > max(container.get_dimensions_wdh()):
                    print("There is one item too big for this container 🙃")
                    is_item_too_big = True
                    break
            
                ##If item was already check for this box, pass OR If the item is too big for the remaining space
                if current_item.name in container.items_names_already_checked or container.get_volume_empty() < current_item.v:
                    continue               
                        
                if put_item(container, current_item):  
                    fitted = True
                    
                    #We flush list
                    container.items_names_already_checked = []
                    break
                else:
                    container.items_names_already_checked.append(current_item.name)
            
            if is_item_too_big:
                break
            
            if not fitted:
                items_not_packed.append(current_item)
                
    return containers     

# Dash

### Figure

In [10]:
def plot_cube_go(xyz, wdh, fig, opacity = 0.6):
    """Plot a cube with dimensions wdh at position xyz
    
    keyword arguments:
    xyz -- Position that ou want plot the cube
    wdh -- tuple of dimensions of the container that you want for the cube (width, depth, height)
    fig -- go.Figure that you want plot in
    opacity -- integer of the cube that you plot"""
    fig.add_mesh3d(
        # 8 vertices of a cube
        x=[xyz[0], xyz[0], xyz[0]+wdh[0], xyz[0]+wdh[0], xyz[0], xyz[0], xyz[0]+wdh[0], xyz[0]+wdh[0]],
        y=[xyz[1], xyz[1]+wdh[1], xyz[1]+wdh[1], xyz[1], xyz[1], xyz[1]+wdh[1], xyz[1]+wdh[1], xyz[1]],
        z=[xyz[2], xyz[2], xyz[2], xyz[2], xyz[2]+wdh[2], xyz[2]+wdh[2], xyz[2]+wdh[2], xyz[2]+wdh[2]],
        # Intensity of each vertex, which will be interpolated and color-coded
        # i, j and k give the vertices of triangles
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
        opacity = opacity,
        )
    return fig

def plot_containers_with_items_go(containers):
    """Plot containers and items inside. Return a list of go.Figure
    
    keyword arguments:
    containers -- list of Container with items inside
    """
    
    figures = []
    for i,container in enumerate(containers):
        fig = go.Figure()
        xyz_cont, wdh_cont= ((0, 0, 0), (container.get_w(), container.get_d(), container.get_h()))
        plot_cube_go(xyz_cont, wdh_cont, fig, 0.2)
        
        container.items.sort(key = lambda item: (item.get_x(),item.get_z(),item.get_y()))
        
        for box in container.items:
            dimensions_box = get_dimensions_item(box, box.rotation)
            wdh = dimensions_box.get('w'), dimensions_box.get('d'), dimensions_box.get('h')
            x = box.get_x()
            y = box.get_y()
            z = box.get_z()
            xyz = (x,y,z)
            plot_cube_go(xyz, wdh, fig, 1)
            
        figures.append(fig)
    return figures


### Function to Dash

In [11]:
def graphique_boites(figures):
    """Render figure with a layout which show the container number
    
    keyword arguments:
    figures -- list of go.Figure"""
    
    graphiques = []
    for i,figure in enumerate(figures): 
        graphiques.append(dcc.Graph(
            id='graph{}'.format(i),
            figure={'data': figure.data, 'layout': {'title': 'Container n°{}'.format(i+1)}}))
    return graphiques

def generate_table(number_items):
    """Generate html.Table with input for dimensions of boxes. There are number_items rows in the Table
    
    keyword arguments:
    number_items -- integer which is the number of type of item"""
    
    header = ["","Width of box", "Depth of box", "Height of box", "Number of this box"]
    id_tags = ["", "width", "depth", "height", "number_item"]

    header_table = [html.Tr([html.Th(col) for col in header])]
    
    content_table = []
    for i in range(number_items):
        ligne = []
        for j, col in enumerate(header):
            if j == 0:
                ligne.append(html.Td(children = "Box n°{}".format(i+1)))
            elif j == len(header)-1: ## For max type of box we take 40 in order to not have a lot of box, it's a demo
                ligne.append(html.Td(children = dcc.Input(id="{}{}".format(id_tags[j],i), value=random.randint(10,20),\
                                                          type='number', min = 0, max = 40)))
            else:
                ligne.append(html.Td(children = dcc.Input(id="{}{}".format(id_tags[j],i), value=random.randint(10,20), \
                                                          type='number', min = 1, max = 30)))
        content_table.append(html.Tr(ligne))
    
    return html.Table(header_table + content_table)

# Dash page

In [12]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H4(children='Container Loading Problem 📦📦', style = {"textAlign":"center"}),
    html.Div(children = [html.P("Here you can insert the features of 4 different types of boxes. These boxes will be sorted in order to fit into containers of 40x40x40."),
                        html.P("The dimensions (width, depth and height) of each type of box must be included between 1 and 30.\
                               The maximum number of boxes for each type is limited to 40.")]),
    generate_table(4),
    html.Button(id = 'submit-button-graphs', n_clicks=0, children='Submit'),
    html.Div(id = 'my-div-graphs')
])


@app.callback(Output('my-div-graphs', 'children'),
              [Input('submit-button-graphs', 'n_clicks')],
              [State("width0", "value"), State("depth0", "value"), State("height0", "value"), State("number_item0", "value"),\
              State("width1", "value"), State("depth1", "value"), State("height1", "value"), State("number_item1", "value"),\
              State("width2", "value"), State("depth2", "value"), State("height2", "value"), State("number_item2", "value"),\
              State("width3", "value"), State("depth3", "value"), State("height3", "value"), State("number_item3", "value")])
def create_graphs(n_cliks, width0, depth0, height0, n_item0, width1, depth1, height1, n_item1,\
                 width2, depth2, height2, n_item2, width3, depth3, height3, n_item3):
    """Create the graphs corresponding to the differents boxes that you want to put in containers of (40,40,40)"""
    
    if n_cliks >=1:
        items_inseres = []
        for i in range(int(float(n_item0))):
            items_inseres.append(Item(width0, depth0, height0, "item_0"))
        for i in range(int(float(n_item1))):
            items_inseres.append(Item(width1, depth1, height1, "item_1"))
        for i in range(int(float(n_item2))):
            items_inseres.append(Item(width2, depth2, height2, "item_2"))
        for i in range(int(float(n_item3))):
            items_inseres.append(Item(width3, depth3, height3, "item_3"))
        containers_to_draw = []
        containers_to_draw = best_fit_3D(items_inseres, (40, 40, 40))
        figures = plot_containers_with_items_go(containers_to_draw)
        
        return graphique_boites(figures)
         
        
        

    


app.run_server(host='127.0.0.1', port=8881, debug=True)

47
35
22
9


61
49
37
25
8
